# DNN approach for maRAPID

Goodsol Lee, Netlab, Seoul National University

## Get datasets

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os

#configuration for gpu usage
conf = tf.ConfigProto()
# you can modify below as you want
#conf.gpu_options.per_process_gpu_memory_fraction = 0.4
#conf.gpu_options.allow_growth = True
os.environ['CUDA_VISIBLE_DEVICES']='0'
print(tf.__version__)

1.12.0


<font size="5">Sample training data, validation data, test data from raw data</font>

In [2]:
node_num = 155
num_RSSI_sample = 25

data_set = np.empty((num_RSSI_sample))
channel_label_set = np.empty((1))
TA_label_set = np.empty((1))

for node_index in range(1,node_num+1):
    print('Get sample from node'+str(node_index))
    file_name = 'data/node-'+str(node_index)+'.txt'
    previous_cellId = -1
    cell_sample = list()
    cell_sample_set = list()
    
    with open(file_name, 'r') as f:
        while 1:
            line = f.readline()
            if not line: break
            parsed_line =line.split(' ')
            
            rssi = parsed_line[9]
            channel_state = parsed_line[8]
            current_cellId = parsed_line[10]
            TA_region = parsed_line[11]
            
            if previous_cellId == current_cellId:
                cell_sample.append([rssi, channel_state, TA_region])
            else:
                previous_cellId = current_cellId
                if cell_sample:
                    cell_sample_set.append(cell_sample)
                    cell_sample = list()
        
        cell_sample_set.append(cell_sample)
        sample_length = 0
        for i in range(len(cell_sample_set)):
            sample_length += len(cell_sample_set[i])-num_RSSI_sample+1
        
        print(sample_length)
        
        temp_data_set = np.zeros((sample_length,num_RSSI_sample))
        temp_channel_label_set = np.zeros((sample_length,1))
        temp_TA_label_set = np.zeros((sample_length,1))
        
        for i in range(len(cell_sample_set)):
            cell_sample = cell_sample_set[i]
            cell_sample = np.array(cell_sample)
            for j in range(len(cell_sample_set[i])-num_RSSI_sample+1):
                temp_data_set[j] = cell_sample[j:j+num_RSSI_sample,0]
                temp_channel_label_set[j] = cell_sample[j+num_RSSI_sample-1,1]
                temp_TA_label_set[j] = cell_sample[j+num_RSSI_sample-1,2] 
              
    if node_index == 1:
        data_set = temp_data_set
        channel_label_set = temp_channel_label_set
        TA_label_set = temp_TA_label_set
    else:
        data_set = np.append(data_set,temp_data_set,axis = 0)
        channel_label_set = np.append(channel_label_set,temp_channel_label_set,axis = 0)
        TA_label_set = np.append(TA_label_set,temp_TA_label_set,axis=0)
    
print('Sampling is completed, sample length: ',data_set.shape[0])

Get sample from node1
25678
Get sample from node2
15553
Get sample from node3
25285
Get sample from node4
18552
Get sample from node5
26527
Get sample from node6
21138
Get sample from node7
30513
Get sample from node8
52905
Get sample from node9
26551
Get sample from node10
21704
Get sample from node11
21477
Get sample from node12
16702
Get sample from node13
33326
Get sample from node14
23333
Get sample from node15
52102
Get sample from node16
34728
Get sample from node17
26327
Get sample from node18
29110
Get sample from node19
22327
Get sample from node20
24527
Get sample from node21
41902
Get sample from node22
40852
Get sample from node23
11776
Get sample from node24
23927
Get sample from node25
27926
Get sample from node26
42306
Get sample from node27
29327
Get sample from node28
20752
Get sample from node29
24151
Get sample from node30
25154
Get sample from node31
35131
Get sample from node32
34127
Get sample from node33
25151
Get sample from node34
20526
Get sample from node35


In [3]:
for i in range(1):
    print('me'+'me')

meme


In [4]:
import random

data_length = data_set.shape[0]
valid_num = 100000
test_num = 100000

#get test set
test_index = random.sample(range(0,data_length-test_num),test_num)
test_data = data_set[test_index]
test_channel_label = channel_label_set[test_index]
test_TA_label = TA_label_set[test_index]

#get training set/validation set
train_data_set = np.delete(data_set,test_index,axis=0)
train_channel_label_set = np.delete(channel_label_set,test_index,axis=0)
train_TA_label_set = np.delete(TA_label_set,test_index,axis=0)
data_length = train_data_set.shape[0]

valid_index = random.sample(range(0,data_length),valid_num)

training_data = train_data_set
training_channel_label = train_channel_label_set
training_TA_label = train_TA_label_set

valid_data = train_data_set[valid_index]
valid_channel_label = train_channel_label_set[valid_index]
valid_TA_label = train_TA_label_set[valid_index]

save_data ={
    'training_data':training_data,
    'training_channel_label':training_channel_label,
    'training_TA_label':training_TA_label,
    
    'valid_data':valid_data,
    'valid_channel_label':valid_channel_label,
    'valid_TA_label':valid_TA_label,
    
    'test_data':test_data,
    'test_channel_label':test_channel_label,
    'test_TA_label':test_TA_label
}

In [5]:
# get name of the data path
data_path = 'data/save_data'+str(num_RSSI_sample)+'.pickle'

In [6]:
# save data
with open(data_path,'wb') as f:
    pickle.dump(save_data,f,pickle.HIGHEST_PROTOCOL)

In [7]:
# restore data
with open(data_path,'rb') as f:
    save = pickle.load(f)
    
    train_dataset = save['training_data']
    train_channel_labels = save['training_channel_label']
    train_labels = save['training_TA_label']
    
    valid_dataset = save['valid_data']
    valid_channel_labels = save['valid_channel_label']
    valid_labels = save['valid_TA_label']
    
    test_dataset = save['test_data']
    test_channel_labels = save['test_channel_label']
    test_labels = save['test_TA_label']
    
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (3952323, 25) (3952323, 1)
Validation set (100000, 25) (100000, 1)
Test set (100000, 25) (100000, 1)


In [8]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, num_RSSI_sample)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    enc.fit(labels)
    labels = enc.transform(labels).toarray()
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Training set (3952323, 25) (3952323, 4)
Validation set (100000, 25) (100000, 4)
Test set (100000, 25) (100000, 4)


/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

In [9]:
sample_size = num_RSSI_sample
num_labels = 4

layer1_output_num = 200
layer2_output_num = 200
layer3_output_num = 200
layer4_output_num = 512
layer5_output_num = 256
num_steps = 1000
batch_size = 100000

graph_gs=tf.Graph()
with graph_gs.as_default():
    tf_dataset_gs=tf.placeholder(tf.float32, shape=(None, sample_size))
    tf_labels_gs=tf.placeholder(tf.float32, shape=(None, num_labels))
    keep_prob = tf.placeholder(tf.float32, shape=(None))
    is_train = tf.placeholder(tf.bool)
    
    lambda_reg = 0.000000000001
        
    #Regularization
    regularizer = tf.contrib.layers.l2_regularizer(scale=lambda_reg)
    initializer = tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)
    #neural network consists of two lines
    dense1 = tf.layers.dense(tf_dataset_gs, layer1_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense1 = tf.nn.dropout(dense1, keep_prob = keep_prob)
    dense1 = tf.layers.batch_normalization(dense1, training=is_train)
    
    dense2 = tf.layers.dense(dense1,layer2_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense2 = tf.nn.dropout(dense2, keep_prob = keep_prob)
    dense2 = tf.layers.batch_normalization(dense2, training=is_train)
    
    dense3 = tf.layers.dense(dense2 ,layer3_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense3 = tf.nn.dropout(dense3, keep_prob = keep_prob)
    dense3 = tf.layers.batch_normalization(dense3, training=is_train)
    
    dense4 = tf.layers.dense(dense3 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense4 = tf.nn.dropout(dense4, keep_prob = keep_prob)
    dense4 = tf.layers.batch_normalization(dense4, training=is_train)
    
    dense5 = tf.layers.dense(dense4 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense5 = tf.nn.dropout(dense5, keep_prob = keep_prob)
    dense5 = tf.layers.batch_normalization(dense5, training=is_train)
    logits_gs = tf.layers.dense(dense2, num_labels, activation=None)
    
    #Loss
    loss_gs = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_labels_gs, logits=logits_gs))
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.005
    
    decay_num =  50*(int)(train_labels.shape[0]/batch_size)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_num, 1, staircase=True)
    # Optimizer
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer_gs = tf.train.AdamOptimizer(learning_rate).minimize(loss_gs, global_step=global_step)
    
    #Predictions for the training
    prediction_gs = tf.nn.softmax(logits_gs)

In [10]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.equal(np.argmax(predictions, 1), np.argmax(labels, 1)))
          / predictions.shape[0])

tf.reset_default_graph()
batch_num = (int)(train_labels.shape[0]/batch_size)

with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        random = np.random.permutation(train_labels.shape[0])
        loss = 0
        for b in range(batch_num):
            batch_data = train_dataset[random[b*batch_size:(b+1)*batch_size]]
            batch_labels = train_labels[random[b*batch_size:(b+1)*batch_size]].astype(float)
            feed_dict_gs = {tf_dataset_gs: batch_data, tf_labels_gs: batch_labels, keep_prob:1, is_train:True}
            _, l_gs, predictions_l = session_gs.run([optimizer_gs, loss_gs, prediction_gs], feed_dict=feed_dict_gs)
            loss += l_gs
        print("Minibatch loss at step %d: %f" % (step, loss))
        feed_dict_val_gs = {tf_dataset_gs: valid_dataset}
        valid_prediction_gs = session_gs.run(prediction_gs, feed_dict={tf_dataset_gs: valid_dataset, tf_labels_gs: valid_labels, keep_prob:1, is_train: False})
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction_gs, valid_labels))
        
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1.0, is_train: False}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))
    saver = tf.train.Saver()
    saver.save(session_gs, "./model_checkpoints/centralized_sample"+str(sample_size))

Initialized
Minibatch loss at step 0: 20.447573
Validation accuracy: 62.1%
Minibatch loss at step 1: 9.152813
Validation accuracy: 62.1%
Minibatch loss at step 2: 8.953965
Validation accuracy: 62.1%
Minibatch loss at step 3: 8.943863
Validation accuracy: 62.1%
Minibatch loss at step 4: 8.921030
Validation accuracy: 62.7%
Minibatch loss at step 5: 8.936125
Validation accuracy: 64.1%
Minibatch loss at step 6: 8.912061
Validation accuracy: 67.3%
Minibatch loss at step 7: 8.907360
Validation accuracy: 69.9%
Minibatch loss at step 8: 8.906238
Validation accuracy: 71.2%
Minibatch loss at step 9: 8.902484
Validation accuracy: 73.8%
Minibatch loss at step 10: 8.913095
Validation accuracy: 74.4%
Minibatch loss at step 11: 8.912536
Validation accuracy: 75.8%
Minibatch loss at step 12: 8.914952
Validation accuracy: 76.7%
Minibatch loss at step 13: 8.896912
Validation accuracy: 77.7%
Minibatch loss at step 14: 8.899108
Validation accuracy: 79.7%
Minibatch loss at step 15: 8.896188
Validation accur

Minibatch loss at step 130: 8.894848
Validation accuracy: 88.0%
Minibatch loss at step 131: 8.885290
Validation accuracy: 85.3%
Minibatch loss at step 132: 8.884370
Validation accuracy: 85.3%
Minibatch loss at step 133: 8.870096
Validation accuracy: 88.0%
Minibatch loss at step 134: 8.880593
Validation accuracy: 88.0%
Minibatch loss at step 135: 8.901769
Validation accuracy: 85.7%
Minibatch loss at step 136: 8.882572
Validation accuracy: 86.6%
Minibatch loss at step 137: 8.884258
Validation accuracy: 86.3%
Minibatch loss at step 138: 8.879420
Validation accuracy: 87.3%
Minibatch loss at step 139: 8.878533
Validation accuracy: 86.7%
Minibatch loss at step 140: 8.886680
Validation accuracy: 85.4%
Minibatch loss at step 141: 8.880703
Validation accuracy: 87.9%
Minibatch loss at step 142: 8.885815
Validation accuracy: 85.2%
Minibatch loss at step 143: 8.870768
Validation accuracy: 87.2%
Minibatch loss at step 144: 8.878618
Validation accuracy: 87.0%
Minibatch loss at step 145: 8.874808
Val

Minibatch loss at step 259: 8.881953
Validation accuracy: 87.7%
Minibatch loss at step 260: 8.873760
Validation accuracy: 88.0%
Minibatch loss at step 261: 8.874078
Validation accuracy: 87.6%
Minibatch loss at step 262: 8.882549
Validation accuracy: 85.9%
Minibatch loss at step 263: 8.874812
Validation accuracy: 86.5%
Minibatch loss at step 264: 8.896572
Validation accuracy: 86.3%
Minibatch loss at step 265: 8.908630
Validation accuracy: 85.8%
Minibatch loss at step 266: 8.878272
Validation accuracy: 87.7%
Minibatch loss at step 267: 8.869312
Validation accuracy: 87.7%
Minibatch loss at step 268: 8.883948
Validation accuracy: 86.6%
Minibatch loss at step 269: 8.884913
Validation accuracy: 87.5%
Minibatch loss at step 270: 8.874199
Validation accuracy: 86.0%
Minibatch loss at step 271: 8.867555
Validation accuracy: 87.8%
Minibatch loss at step 272: 8.874266
Validation accuracy: 88.4%
Minibatch loss at step 273: 8.876756
Validation accuracy: 88.5%
Minibatch loss at step 274: 8.877715
Val

Minibatch loss at step 388: 8.876658
Validation accuracy: 88.5%
Minibatch loss at step 389: 8.863392
Validation accuracy: 88.6%
Minibatch loss at step 390: 8.866917
Validation accuracy: 88.2%
Minibatch loss at step 391: 8.865075
Validation accuracy: 88.4%
Minibatch loss at step 392: 8.874100
Validation accuracy: 88.0%
Minibatch loss at step 393: 8.877015
Validation accuracy: 88.7%
Minibatch loss at step 394: 8.863924
Validation accuracy: 88.4%
Minibatch loss at step 395: 8.863243
Validation accuracy: 88.8%
Minibatch loss at step 396: 8.861362
Validation accuracy: 89.0%
Minibatch loss at step 397: 8.874074
Validation accuracy: 88.7%
Minibatch loss at step 398: 8.867092
Validation accuracy: 88.3%
Minibatch loss at step 399: 8.881968
Validation accuracy: 89.2%
Minibatch loss at step 400: 8.877333
Validation accuracy: 89.1%
Minibatch loss at step 401: 8.882063
Validation accuracy: 88.7%
Minibatch loss at step 402: 8.875565
Validation accuracy: 87.8%
Minibatch loss at step 403: 8.870642
Val

Minibatch loss at step 517: 8.865342
Validation accuracy: 89.2%
Minibatch loss at step 518: 8.864526
Validation accuracy: 89.3%
Minibatch loss at step 519: 8.877947
Validation accuracy: 88.9%
Minibatch loss at step 520: 8.867218
Validation accuracy: 89.1%
Minibatch loss at step 521: 8.868224
Validation accuracy: 89.1%
Minibatch loss at step 522: 8.867181
Validation accuracy: 89.1%
Minibatch loss at step 523: 8.870403
Validation accuracy: 89.0%
Minibatch loss at step 524: 8.873435
Validation accuracy: 89.1%
Minibatch loss at step 525: 8.860965
Validation accuracy: 89.0%
Minibatch loss at step 526: 8.866869
Validation accuracy: 89.1%
Minibatch loss at step 527: 8.860522
Validation accuracy: 89.1%
Minibatch loss at step 528: 8.867268
Validation accuracy: 88.8%
Minibatch loss at step 529: 8.861529
Validation accuracy: 89.1%
Minibatch loss at step 530: 8.868606
Validation accuracy: 89.2%
Minibatch loss at step 531: 8.862040
Validation accuracy: 89.2%
Minibatch loss at step 532: 8.865863
Val

Minibatch loss at step 646: 8.867437
Validation accuracy: 88.8%
Minibatch loss at step 647: 8.863062
Validation accuracy: 89.0%
Minibatch loss at step 648: 8.869162
Validation accuracy: 88.9%
Minibatch loss at step 649: 8.862691
Validation accuracy: 89.0%
Minibatch loss at step 650: 8.864006
Validation accuracy: 89.0%
Minibatch loss at step 651: 8.861938
Validation accuracy: 89.3%
Minibatch loss at step 652: 8.868344
Validation accuracy: 89.1%
Minibatch loss at step 653: 8.867041
Validation accuracy: 89.2%
Minibatch loss at step 654: 8.861537
Validation accuracy: 89.3%
Minibatch loss at step 655: 8.857241
Validation accuracy: 89.1%
Minibatch loss at step 656: 8.865775
Validation accuracy: 89.3%
Minibatch loss at step 657: 8.872579
Validation accuracy: 89.1%
Minibatch loss at step 658: 8.864996
Validation accuracy: 89.3%
Minibatch loss at step 659: 8.863495
Validation accuracy: 88.9%
Minibatch loss at step 660: 8.866700
Validation accuracy: 89.3%
Minibatch loss at step 661: 8.863105
Val

Minibatch loss at step 775: 8.862507
Validation accuracy: 89.3%
Minibatch loss at step 776: 8.860339
Validation accuracy: 89.2%
Minibatch loss at step 777: 8.864556
Validation accuracy: 89.3%
Minibatch loss at step 778: 8.863983
Validation accuracy: 89.3%
Minibatch loss at step 779: 8.864160
Validation accuracy: 89.3%
Minibatch loss at step 780: 8.860457
Validation accuracy: 89.0%
Minibatch loss at step 781: 8.865002
Validation accuracy: 89.3%
Minibatch loss at step 782: 8.864102
Validation accuracy: 89.3%
Minibatch loss at step 783: 8.861045
Validation accuracy: 89.3%
Minibatch loss at step 784: 8.865319
Validation accuracy: 89.3%
Minibatch loss at step 785: 8.862155
Validation accuracy: 89.1%
Minibatch loss at step 786: 8.866470
Validation accuracy: 89.3%
Minibatch loss at step 787: 8.860773
Validation accuracy: 89.3%
Minibatch loss at step 788: 8.859975
Validation accuracy: 89.1%
Minibatch loss at step 789: 8.864376
Validation accuracy: 89.1%
Minibatch loss at step 790: 8.869871
Val

Minibatch loss at step 904: 8.872505
Validation accuracy: 89.3%
Minibatch loss at step 905: 8.862423
Validation accuracy: 89.1%
Minibatch loss at step 906: 8.862557
Validation accuracy: 89.3%
Minibatch loss at step 907: 8.864525
Validation accuracy: 89.3%
Minibatch loss at step 908: 8.858666
Validation accuracy: 89.0%
Minibatch loss at step 909: 8.861141
Validation accuracy: 89.3%
Minibatch loss at step 910: 8.860847
Validation accuracy: 89.3%
Minibatch loss at step 911: 8.859454
Validation accuracy: 89.2%
Minibatch loss at step 912: 8.865905
Validation accuracy: 89.2%
Minibatch loss at step 913: 8.866002
Validation accuracy: 89.2%
Minibatch loss at step 914: 8.860545
Validation accuracy: 89.3%
Minibatch loss at step 915: 8.861952
Validation accuracy: 89.2%
Minibatch loss at step 916: 8.864741
Validation accuracy: 89.3%
Minibatch loss at step 917: 8.857831
Validation accuracy: 89.2%
Minibatch loss at step 918: 8.862714
Validation accuracy: 89.1%
Minibatch loss at step 919: 8.868084
Val

In [93]:
with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))

Test accuracy: 52.5%
